In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv("D:\\Data_Science\\Datasets\\Kaggle_data\\fake-news\\train.csv")
test_df = pd.read_csv("D:\\Data_Science\\Datasets\\Kaggle_data\\fake-news\\test.csv")

In [3]:
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
train_df.shape

(20800, 5)

In [5]:
train_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
train_df = train_df.dropna()


In [7]:
train_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
train_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
y = train_df['label']
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
voc_size = 5000

In [13]:
massege = x.copy()

NameError: name 'x' is not defined

In [ ]:
massege['title'][1]

In [ ]:
massege

In [ ]:
massege.reset_index(inplace=True)

In [ ]:
massege

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
len(massege)

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(massege)):
    review = re.sub('[^a-zA-Z]', ' ', massege['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
corpus[1]

In [ ]:
onehot_repr = [one_hot(words,voc_size)for words in corpus] 
onehot_repr

In [ ]:
corpus[2]

In [ ]:
onehot_repr[2]

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

In [ ]:
embedded_docs[2]

# LSTM Networks

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
#model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
len(embedded_docs),y.shape

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
import pickle
clf = LogisticRegressionCV(cv=5, scoring='accuracy', random_state=0, n_jobs=-1, verbose=3, max_iter=300).fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = clf.predict(X_test)
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn import metrics
import numpy as np
import itertools

cm = metrics.confusion_matrix(y_test,y_pred)
plot_confusion_matrix(cm, classes=['FAKE', 'REAL'])